# Deep Learning

## preprocessing training dataset

In [302]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
input_file_dir = "/Users/ninglee/Documents/IntrutionDection/datasets"
train_file_name = "kddcup.data_10_percent.txt"
test_file_name = "corrected.txt"
header_file_name = "header.txt"
train_files = os.path.join(input_file_dir, train_file_name)
test_files = os.path.join(input_file_dir, test_file_name)
header_files = os.path.join(input_file_dir, header_file_name)

In [303]:
with open(header_files, 'r') as f:
    header = f.readline().strip().split(',')
train_dataset = pd.read_csv(train_files)
test_dataset = pd.read_csv(test_files)
train_dataset.columns = header
test_dataset.columns = header

In [304]:
train_dataset_size = train_dataset.shape[0]
test_dataset_size = test_dataset.shape[0]
train_dataset = pd.concat([train_dataset, test_dataset], axis=0)
print train_dataset_size, test_dataset_size

494020 311028


In [305]:
def labels_map(label):
    label = str(label).split('.')[0]
    if label == 'normal':
        return 0
    if label in ['ipsweep', 'mscan', 'nmap', 'portsweep', 'saint', 'satan']: #PROBE
        return 1
    if label in ['apache2', 'back', 'land', 'mailbomb', 'neptune', 'pod', 'processtable', 'smurf', 'teardrop', 'udpstorm']: #DOS
        return 2
    if label in ['buffer_overflow', 'httptunnel', 'loadmodule', 'perl', 'ps', 'rootkit', 'sqlattack', 'xterm']: #U2R
        return 3
    if label in ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'named', 'phf', 'sendmail', 'snmpgetattack', 'snmpguess', 'spy', 'warezclient', 'warezmaster', 'worm', 'xlock', 'xsnoop']: #R2L
        return 4
    
train_dataset['labels'] = train_dataset['labels'].apply(labels_map)
labels_dummies = pd.get_dummies(train_dataset['labels'], prefix='label')
train_dataset = pd.concat([train_dataset,labels_dummies], axis=1)

In [306]:
train_dataset.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels,label_0,label_1,label_2,label_3,label_4
0,0,tcp,http,SF,239,486,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
1,0,tcp,http,SF,235,1337,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
2,0,tcp,http,SF,219,1337,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
3,0,tcp,http,SF,217,2032,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0


In [307]:
protocal_type_dummies = pd.get_dummies(train_dataset.protocol_type, prefix='protocol_type')
service_dummies = pd.get_dummies(train_dataset.service, prefix='service')
flag_dummies = pd.get_dummies(train_dataset.flag, prefix='flag')
train_dataset = pd.concat([train_dataset, protocal_type_dummies, service_dummies, flag_dummies], axis=1)

In [308]:
max1 = train_dataset.src_bytes.max(); min1 = train_dataset.src_bytes.min();
max2 = train_dataset.dst_bytes.max(); min2 = train_dataset.dst_bytes.min();
train_dataset['src_bytes_norm'] = (train_dataset.src_bytes - min1) / float(max1 - min1)
train_dataset['dst_bytes_norm'] = (train_dataset.dst_bytes - min2) / float(max2 - min2)

In [309]:
train_dataset = train_dataset.drop(['protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes','labels'], axis=1)

In [310]:
train_dataset = train_dataset.astype('float')
# train_dataset = (train_dataset - train_dataset.min()) / (train_dataset.max() - train_dataset.min())

## preprocessing test dataset

In [311]:
sub_train_dataset = train_dataset.iloc[train_dataset_size:, :].sample(n=50000)
sub_test_dataset = train_dataset.iloc[:train_dataset_size, :].sample(n=10000)
sub_train_labels = sub_train_dataset[['label_0', 'label_1', 'label_2', 'label_3', 'label_4']]
sub_test_labels = sub_test_dataset[['label_0', 'label_1', 'label_2', 'label_3', 'label_4']]
sub_train_dataset.drop(['label_0', 'label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)
sub_test_dataset.drop(['label_0', 'label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)

In [312]:
test_dataset = train_dataset.iloc[train_dataset_size:,:]
train_dataset = train_dataset.iloc[:train_dataset_size, :]
train_labels = train_dataset[['label_0', 'label_1', 'label_2', 'label_3', 'label_4']]
test_labels = test_dataset[['label_0', 'label_1', 'label_2', 'label_3', 'label_4']]
train_dataset.drop(['label_0', 'label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)
test_dataset.drop(['label_0', 'label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)

In [313]:
print train_dataset.shape, train_labels.shape
print test_dataset.shape, test_labels.shape

(494020, 119) (494020, 5)
(311028, 119) (311028, 5)


In [314]:
print sub_train_dataset.shape, sub_train_labels.shape
print sub_test_dataset.shape, sub_test_labels.shape

(50000, 119) (50000, 5)
(10000, 119) (10000, 5)


In [315]:
sub_train_dataset.describe()

,duration,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,...,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,src_bytes_norm,dst_bytes_norm
count,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.0,...,50000.0,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000
mean,19.496860,0.000020,0.000640,0.00006,0.017200,0.002560,0.17576,0.008120,0.000200,0.0,...,0.0,0.002700,0.058080,0.000160,0.000040,0.000860,0.800780,0.000160,2.360905e-06,0.000136
std,475.794869,0.004472,0.038466,0.01000,0.529707,0.051706,0.38062,0.751895,0.014141,0.0,...,0.0,0.051892,0.233897,0.012648,0.006324,0.029313,0.399418,0.012648,3.775728e-05,0.002167
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.514331e-07,0.000000
50%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,7.499542e-07,0.000000
75%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.488371e-06,0.000000
max,53771.000000,1.000000,3.000000,2.00000,101.000000,3.000000,1.00000,165.000000,1.000000,0.0,...,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.605842e-03,0.359027


## model1: MLP

In [316]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [348]:
batch_size = 200
feature_size = 119
num_labels = 5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
#   tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(sub_test_dataset.values[:,:])
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([feature_size, 1024]))
  output_weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  weights = [hidden_weights, output_weights]
  hidden_biases = tf.Variable(tf.zeros([1024]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  biases = [hidden_biases, output_biases]
  
  # Training computation.
  hidden_layer = tf.add(tf.matmul(tf_train_dataset, tf.cast(weights[0], tf.float32)), biases[0])
  hidden_layer = tf.nn.relu(hidden_layer)
  logits = tf.add(tf.matmul(hidden_layer, tf.cast(weights[1], tf.float32)), biases[1])
#   logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  l2Loss = tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])
  loss = loss + 5*l2Loss
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  # ValidPrediction
#   temp1 = tf.add(tf.matmul(tf_valid_dataset, weights[0]), biases[0])
#   hidden_temp = tf.nn.relu(temp1)
#   temp2 = tf.add(tf.matmul(hidden_temp,weights[1]), biases[1])
#   valid_prediction = tf.nn.softmax(temp2)

  # TestPrediction
  temp3 = tf.add(tf.matmul(tf.cast(tf_test_dataset, tf.float32), weights[0]), biases[0])
  hidden_temp2 = tf.nn.relu(temp3)
  temp4 = tf.add(tf.matmul(tf.cast(hidden_temp2, tf.float32), weights[1]), biases[1])
  test_prediction = tf.nn.softmax(temp4)

In [349]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (sub_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = sub_train_dataset.iloc[offset:(offset + batch_size), :].values[:,:]
    batch_labels = sub_train_labels.iloc[offset:(offset + batch_size), :].values[:,:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 300 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#       print("Validation accuracy: %.1f%%" % accuracy(
#         valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), sub_test_labels.values[:,:]))

Initialized
Minibatch loss at step 0: 249257.203125
Minibatch accuracy: 38.0%
Test accuracy: 78.5%
Minibatch loss at step 300: 12029.495117
Minibatch accuracy: 90.0%
Test accuracy: 87.3%
Minibatch loss at step 600: 592.334595
Minibatch accuracy: 93.5%
Test accuracy: 92.8%
Minibatch loss at step 900: 29.545288
Minibatch accuracy: 91.5%
Test accuracy: 95.0%
Minibatch loss at step 1200: 1.924597
Minibatch accuracy: 90.0%
Test accuracy: 94.7%
Minibatch loss at step 1500: 0.602791
Minibatch accuracy: 87.5%
Test accuracy: 95.4%
Minibatch loss at step 1800: 0.432140
Minibatch accuracy: 90.5%
Test accuracy: 95.0%
Minibatch loss at step 2100: 0.496623
Minibatch accuracy: 92.0%
Test accuracy: 95.0%
Minibatch loss at step 2400: 0.487768
Minibatch accuracy: 89.0%
Test accuracy: 94.9%
Minibatch loss at step 2700: 0.437116
Minibatch accuracy: 93.0%
Test accuracy: 95.1%
Minibatch loss at step 3000: 0.543201
Minibatch accuracy: 88.0%
Test accuracy: 95.0%
Minibatch loss at step 3300: 0.495481
Minibatch

## model2: CNN

In [12]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [13]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28, 1), (200000, 10))
('Validation set', (10000, 28, 28, 1), (10000, 10))
('Test set', (10000, 28, 28, 1), (10000, 10))


In [14]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [17]:
batch_size = 16 #一次训练16个样本吗？不太清楚  bath_size上的strides=1 应该是一次处理一个样本
patch_size = 5 #相当于是kernel的大小
depth = 16 #最后输出的大小
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # kernel里面的值随机初始化吗？？？
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth])) # biases初始化为0
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) # 下一次biases又变成1？？？
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    """
    ninglee 标注
    conv2d(
        input,
        filter,
        strides,
        padding,
        use_cudnn_on_gpu=None,
        data_format=None, 表示输入输出的格式，默认[batch, height, width, channels]
        name=None
    )
    input: 4-D [batch, in_height, in_width, in_channels]
    filter/kernel: 4-D [filter_height, filter_width, in_channels, out_channels]
    strides: 4-D 4个维度的步幅
    padding: string "SAME" "VALID"
    """
    # 第一层卷积变换
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME') #strides = 2 ninglee标注
    hidden = tf.nn.relu(conv + layer1_biases)# 卷积完了之后还要加上biases然后经过relu激活函数处理才算完
    # 第二层卷积变换
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # 全连接层，同样也要经过relu激活
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    # 最后的输出层，使用softmax激活函数
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.702314
Minibatch accuracy: 0.0%
Validation accuracy: 9.9%
Minibatch loss at step 50: 2.009227
Minibatch accuracy: 25.0%
Validation accuracy: 50.8%
Minibatch loss at step 100: 1.051008
Minibatch accuracy: 62.5%
Validation accuracy: 64.7%
Minibatch loss at step 150: 0.693865
Minibatch accuracy: 75.0%
Validation accuracy: 74.9%
Minibatch loss at step 200: 1.195843
Minibatch accuracy: 68.8%
Validation accuracy: 77.5%
Minibatch loss at step 250: 0.924027
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 300: 0.626783
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 350: 1.750046
Minibatch accuracy: 43.8%
Validation accuracy: 79.4%
Minibatch loss at step 400: 1.061372
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.848145
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 500: 0.397961
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Min

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [18]:
batch_size = 16 #一次训练16个样本吗？不太清楚  bath_size上的strides=1 应该是一次处理一个样本
patch_size = 5 #相当于是kernel的大小
depth = 16 #最后输出的大小
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # kernel里面的值随机初始化吗？？？
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth])) # biases初始化为0
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) # 下一次biases又变成1？？？
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    """
    ninglee 标注
    conv2d(
        input,
        filter,
        strides,
        padding,
        use_cudnn_on_gpu=None,
        data_format=None, 表示输入输出的格式，默认[batch, height, width, channels] NHWC
        name=None
    )
    input: 4-D [batch, in_height, in_width, in_channels]
    filter/kernel: 4-D [filter_height, filter_width, in_channels, out_channels]
    strides: 4-D 4个维度的步幅
    padding: string "SAME" "VALID"
    
    max_pool(
        value,
        ksize,
        strides,
        padding,
        data_format='NHWC',
        name=None
    )
    value: 4-D [batch, height, width, channels]
    ksize: list of ints, 长度至少为4
    strides: list of ints, 长度至少为4
    padding: string, "VALID", "SAME"
    
    """
    # 第一层卷积变换
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') 
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    # 第二层卷积变换
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # 注意：一定是等到relu激活函数之后再池化
    hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    # 全连接层，同样也要经过relu激活
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    # 最后的输出层，使用softmax激活函数
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [19]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3.140602
Minibatch accuracy: 6.2%
Validation accuracy: 9.1%
Minibatch loss at step 50: 2.370754
Minibatch accuracy: 12.5%
Validation accuracy: 23.0%
Minibatch loss at step 100: 1.429823
Minibatch accuracy: 43.8%
Validation accuracy: 41.6%
Minibatch loss at step 150: 0.721467
Minibatch accuracy: 75.0%
Validation accuracy: 71.3%
Minibatch loss at step 200: 1.137816
Minibatch accuracy: 62.5%
Validation accuracy: 76.9%
Minibatch loss at step 250: 0.949122
Minibatch accuracy: 68.8%
Validation accuracy: 79.8%
Minibatch loss at step 300: 0.557121
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 350: 1.618942
Minibatch accuracy: 50.0%
Validation accuracy: 79.9%
Minibatch loss at step 400: 0.677687
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.773237
Minibatch accuracy: 75.0%
Validation accuracy: 81.4%
Minibatch loss at s

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [51]:
batch_size = 16*4 #一次训练16个样本吗？不太清楚  bath_size上的strides=1 应该是一次处理一个样本
patch_size = 5  #相当于是kernel的大小
depth = 16 #最后输出的大小
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth])) # biases初始化为0
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) # 下一次biases又变成1？？？
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # 第一层卷积变换
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') 
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    # 第二层卷积变换
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # 注意：一定是等到relu激活函数之后再池化
    hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    # 全连接层，同样也要经过relu激活
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    # 最后的输出层，使用softmax激活函数
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
#   l2_loss = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer3_weights)+ tf.nn.l2_loss(layer4_weights)
  #loss = loss + 0.01*l2_loss
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100,0.96,staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
#   optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [52]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3.091755
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.000209
Minibatch accuracy: 65.6%
Validation accuracy: 76.5%
Minibatch loss at step 100: 0.642079
Minibatch accuracy: 82.8%
Validation accuracy: 79.6%
Minibatch loss at step 150: 0.576964
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 200: 0.840319
Minibatch accuracy: 70.3%
Validation accuracy: 81.9%
Minibatch loss at step 250: 0.568166
Minibatch accuracy: 84.4%
Validation accuracy: 83.3%
Minibatch loss at step 300: 0.665424
Minibatch accuracy: 78.1%
Validation accuracy: 84.1%
Minibatch loss at step 350: 0.382421
Minibatch accuracy: 90.6%
Validation accuracy: 84.4%
Minibatch loss at step 400: 0.497308
Minibatch accuracy: 82.8%
Validation accuracy: 84.8%
Minibatch loss at step 450: 0.724603
Minibatch accuracy: 76.6%
Validation accuracy: 84.3%
Minibatch loss at

In [ ]:
# Result
* start_learn_rate = 0.1
* decay_step = 50
* decay_rate = 0.96
* l2_loss = False
Minibatch loss at step 950: 0.574295
Minibatch accuracy: 81.2%
Validation accuracy: 86.2%
Minibatch loss at step 1000: 0.738783
Minibatch accuracy: 75.0%
Validation accuracy: 86.3%
Test accuracy: 92.4%

* batch_size = 16 * 4
* 还在长
Minibatch loss at step 950: 0.728797
Minibatch accuracy: 82.8%
Validation accuracy: 87.0%
Minibatch loss at step 1000: 0.506990
Minibatch accuracy: 81.2%
Validation accuracy: 87.3%
Test accuracy: 93.1%

* num_steps = 2001
Minibatch loss at step 1950: 0.301008
Minibatch accuracy: 87.5%
Validation accuracy: 88.4%
Minibatch loss at step 2000: 0.313316
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Test accuracy: 94.4%
    
* other ideas
* 在全连层后面再增加一个全连接层，神经元数量设置为它的一半
* 尝试把卷积映射的输出层数降低试试 depth
* 卷积层增加一层？